In [1]:
import httpx
import random
import pandas as pd
import time
import json

In [2]:
csv_file = "./gpt/noticias-2023-05-31.csv"
data = pd.read_csv(csv_file)

In [3]:
TIMEOUT = 300 # Tiempo maximo que esperamos la respuesta de la API (en segundos)

In [4]:
# Auxiliares
def tiempo_de_ejecucion(inicio):
    fin = time.time()
    tiempo_ejecucion = fin - inicio
    print(f"Tiempo de ejecución: {tiempo_ejecucion} segundos")

def print_resultados(data):
    for doc in data:
        print(f"Location: {doc['location']}, ({doc['lat']}, {doc['lng']})")

In [19]:
docs = [] # lista para guardar los documentos obtenidos del csv, en formato JSON
n_noticias = 1
csv_size = data.shape[0]
#Toma noticias random de un csv
for i in range(n_noticias):
    top = csv_size - i
    random_index = random.randint(0, top)
    new = data.iloc[random_index]
    doc = {
        "id": random_index,
        "title": new["title"],
        "date": new["date"],
        "text": new["text"],
        "url": new["url"]
    }
    docs.append(doc)

In [20]:
for doc in docs:
    titulo = str(doc['title'][:50])
    print(f"Fila: {doc['id']}\t Titulo: {titulo}...")

# Para guardar el doc en un .json
with open('doc.json', 'w') as jf: 
    # n_noticias > 1
    json.dump(docs, jf, ensure_ascii=False, indent=4)
    # n_noticias = 1
    #json.dump(docs[0], jf, ensure_ascii=False, indent=4)

Fila: 16	 Titulo: Gobierno llega a acuerdo con Grupo México, anuncia...


In [7]:
# URL de la API
url = "http://146.83.216.166:5001/geolocalize"
# Realiza una solicitud POST a la API con la lista de documentos
async def enviar_documentos():
    # Ajustar el timeout en caso de demora, actualmente 300 segundos
    async with httpx.AsyncClient(timeout=TIMEOUT) as client:
        inicio = time.time()
        response = await client.post(url, json=docs) # json=doc, es lo que se envia a la API

    if response.status_code == 200: # Respuesta buena
        tiempo_de_ejecucion(inicio)
        data = response.json()
        print(f"Respuesta de la API:")
        print_resultados(data)
    else:
        tiempo_de_ejecucion(inicio)
        print(f"Error al llamar a la API. Código de estado: {response.status_code}")
        print("Respuesta de la API:", response.text)

In [14]:
# Función principal para ejecutar el código
await enviar_documentos()

Tiempo de ejecución: 0.1662611961364746 segundos
Error al llamar a la API. Código de estado: 406
Respuesta de la API: {"detail":"Se permite un máximo de 1 documento a procesar por petición"}
